In [180]:
from crim_intervals import *
import pandas as pd
import ast
import matplotlib
from itertools import tee, combinations
import numpy as np

In [246]:
# THIS IS DEV COPY for use with CLOSE/EXACT Matches

import numpy as np

# Converts lists to tuples

def lists_to_tuples_a(el):
    if isinstance(el, list):
        return tuple(el)
    else:
        return el

# Filters for the length of the Presentation Type in the Classifier

def limit_offset_size(array, limit):
    under_limit = np.cumsum(array) <= limit
    return array[: sum(under_limit)]


# Gets the the list of offset differences for each group 

def get_offset_difference_list_a(group):
    # if we do sort values as part of the func call, then we don't need this first line
    group = group.sort_values("start_offset")
    group["next_offset"] = group.start_offset.shift(-1)
    offset_difference_list = (group.next_offset - group.start_offset).dropna().tolist()
    return offset_difference_list

# The classifications are done here

def classify_offsets_a(offset_difference_list):
    """
    Put logic for classifying an offset list here
    """
    #
    offset_difference_list = limit_offset_size(offset_difference_list, 5000)


    
    alt_list = offset_difference_list[::2]
    
    
    if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
        return ("PEN", offset_difference_list)
    elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
        return ("ID", offset_difference_list)
    elif len(offset_difference_list) >= 1:
        return ("Fuga", offset_difference_list)
    else: 
        return ("Singleton", offset_difference_list)

   
# adds predicted type, offsets and entry numbers to the results

def predict_type_a(group):
    offset_differences = get_offset_difference_list_a(group)
    predicted_type, offsets = classify_offsets_a(offset_differences)

    group["predicted_type"] = [predicted_type for i in range(len(group))]
    group["offset_diffs"] = [offsets for i in range(len(group))]
    group["entry_number"] = [i + 1 for i in range(len(group))]

    return group

In [247]:
df = pd.read_csv(('ave_test_set.csv'), index_col=0)

df = df.drop(columns=["ema", "ema_url", "end_measure", "end_beat"])



In [248]:
classified = df.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified["group_number"] = classified.groupby('pattern_generating_match').ngroup()
classified = classified[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

classified.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.
classified.head(25)

g = classified.sort_values(["group_number", "entry_number"])
g.head(50)


,group_number,pattern_generating_match,pattern_matched,part,start_measure,start_beat,entry_number,sum_durs,offset_diffs,predicted_type
18,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],19,1.0,1,14.0,"[14.0, 58.0, 456.0]",Fuga
19,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],20,4.0,2,18.0,"[14.0, 58.0, 456.0]",Fuga
21,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Tenor,28,1.0,3,10.0,"[14.0, 58.0, 456.0]",Fuga
20,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],85,1.0,4,20.0,"[14.0, 58.0, 456.0]",Fuga
73,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,42,1.0,1,22.0,"[0.0, 258.0, 302.0]",Fuga
71,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",[Superius],42,1.0,2,24.0,"[0.0, 258.0, 302.0]",Fuga
74,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,74,2.0,3,10.0,"[0.0, 258.0, 302.0]",Fuga
72,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Altus,106,1.0,4,20.0,"[0.0, 258.0, 302.0]",Fuga
177,2,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,95,1.0,1,24.0,"[48.0, 60.0, 8.0]",Fuga
178,2,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,99,1.0,2,24.0,"[48.0, 60.0, 8.0]",Fuga


In [185]:
def add_previous_off(df_grouped):
    df_grouped["pre_off"] = df_grouped["start_offset"].shift(1)
    return df_grouped

In [211]:
def add_previous_off_1(classified):
    classified["pre_off"] = classified["start_offset"].shift(1)
    return classified

In [186]:
df_grouped.head()

,pattern_generating_match,pattern_matched,piece_title,part,start_offset,end_offset,note_durations,sum_durs
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
5,"(-2, -2, -2, 2, -2)","[-2, -2, -2, 2, -2]",Ave Maria,[Superius],56.0,74.0,"[6.0, 2.0, 4.0, 4.0, 2.0, 2.0]",20.0
...,...,...,...,...,...,...,...,...
210,"(-2, -2, -2, -2, 2)","[-2, -2, -2, -2, 2]",Ave Maria,Bassus,1092.0,1107.0,"[4.0, 3.0, 1.0, 4.0, 3.0, 1.0]",16.0
102,"(2, 2, 2, -2, 2)","[2, 2, 2, -2, 2]",Ave Maria,Tenor,1092.0,1112.0,"[4.0, 4.0, 4.0, 6.0, 2.0, 4.0]",24.0
106,"(2, 2, -2, -2, -2)","[2, 2, -2, -2, -2]",Ave Maria,Altus,1130.0,1139.0,"[2.0, 2.0, 3.0, 1.0, 1.0, 1.0]",10.0
111,"(2, -2, -2, -2, 2)","[2, -2, -2, -2, 2]",Ave Maria,Bassus,1172.0,1180.0,"[2.0, 3.0, 1.0, 1.0, 1.0, 4.0]",12.0


In [221]:
df_grouped = df.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched")
# new.head()
# for col_name, data in grouped:
#     grouped["pre_off"] = grouped["start_offset"].shift(1)
# grouped
	# print("pattern_matched",col_name, "part:",data)
# for n, group in new:
#     group["previous_offset"] = group["start_offset"].shift(1)
#     # print(new)
# return group

df_grouped.head(35)

,pattern_generating_match,pattern_matched,piece_title,part,start_offset,end_offset,note_durations,sum_durs
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
5,"(-2, -2, -2, 2, -2)","[-2, -2, -2, 2, -2]",Ave Maria,[Superius],56.0,74.0,"[6.0, 2.0, 4.0, 4.0, 2.0, 2.0]",20.0
...,...,...,...,...,...,...,...,...
155,"(2, -3, 2, -2, -2)","[2, -3, 2, -2, -2]",Ave Maria,Bassus,1168.0,1178.0,"[2.0, 2.0, 2.0, 3.0, 1.0, 1.0]",11.0
35,"(2, -2, 1, -2, 2)","[2, -2, 1, -2, 2]",Ave Maria,Altus,1168.0,1180.0,"[2.0, 3.0, 1.0, 4.0, 2.0, 8.0]",20.0
163,"(-3, 2, -2, -2, -2)","[-3, 2, -2, -2, -2]",Ave Maria,Bassus,1170.0,1179.0,"[2.0, 2.0, 3.0, 1.0, 1.0, 1.0]",10.0
111,"(2, -2, -2, -2, 2)","[2, -2, -2, -2, 2]",Ave Maria,Bassus,1172.0,1180.0,"[2.0, 3.0, 1.0, 1.0, 1.0, 4.0]",12.0


In [234]:
df_par = df_grouped.apply(add_previous_off)
df_par["diff"] = df_par["start_offset"] - df_par["pre_off"]

df_par
sel = df_par[df_par['diff'] != 0]
sel.head(50)


,pattern_generating_match,pattern_matched,piece_title,part,start_offset,end_offset,note_durations,sum_durs,pre_off,diff
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,NaN,NaN
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,0.0,16.0
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,16.0,16.0
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,32.0,16.0
5,"(-2, -2, -2, 2, -2)","[-2, -2, -2, 2, -2]",Ave Maria,[Superius],56.0,74.0,"[6.0, 2.0, 4.0, 4.0, 2.0, 2.0]",20.0,NaN,NaN
6,"(-2, -2, -2, 2, -2)","[-2, -2, -2, 2, -2]",Ave Maria,Altus,72.0,91.0,"[6.0, 2.0, 4.0, 4.0, 3.0, 1.0]",20.0,56.0,16.0
7,"(-2, -2, -2, 2, -2)","[-2, -2, -2, 2, -2]",Ave Maria,Tenor,88.0,106.0,"[6.0, 2.0, 4.0, 4.0, 2.0, 2.0]",20.0,72.0,16.0
117,"(-2, -2, -2, 2, 2)","[-2, -2, -2, 2, 2]",Ave Maria,Bassus,104.0,126.0,"[6.0, 2.0, 4.0, 4.0, 6.0, 2.0]",24.0,NaN,NaN
9,"(1, 1, 2, 2, -3)","[1, 1, 2, 2, -3]",Ave Maria,[Superius],124.0,144.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 3.0]",23.0,NaN,NaN
13,"(2, 2, -3, -2, -2)","[2, 2, -3, -2, -2]",Ave Maria,[Superius],132.0,148.0,"[4.0, 4.0, 4.0, 3.0, 1.0, 2.0]",18.0,NaN,NaN


In [240]:
classified2 = sel.applymap(lists_to_tuples_a).groupby("pattern_matched").apply(predict_type_a)
classified2["group_number"] = classified2.groupby('pattern_generating_match').ngroup()
classified2 = classified[["piece_title", "group_number", "pattern_generating_match", "pattern_matched", "part", "start_offset", "entry_number", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

# classified2.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.
classified2.head(25)

g = classified2.sort_values(["group_number", "entry_number"])
g.head(50)


,piece_title,group_number,pattern_generating_match,pattern_matched,part,start_offset,entry_number,sum_durs,offset_diffs,predicted_type
18,Ave Maria,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],144.0,1,14.0,"[14.0, 58.0]",Fuga
19,Ave Maria,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],158.0,2,18.0,"[14.0, 58.0]",Fuga
21,Ave Maria,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Tenor,216.0,3,10.0,"[14.0, 58.0]",Fuga
20,Ave Maria,0,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],672.0,4,20.0,"[14.0, 58.0]",Fuga
73,Ave Maria,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,328.0,1,22.0,[0.0],ID
71,Ave Maria,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",[Superius],328.0,2,24.0,[0.0],ID
74,Ave Maria,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,586.0,3,10.0,[0.0],ID
72,Ave Maria,1,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Altus,888.0,4,20.0,[0.0],ID
177,Ave Maria,2,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,756.0,1,24.0,[48.0],ID
178,Ave Maria,2,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,804.0,2,24.0,[48.0],ID


In [219]:
# cl_par = classified.apply(add_previous_off_1)
# cl_par
classified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 8
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   piece_title               194 non-null    object 
 1   group_number              194 non-null    int64  
 2   pattern_generating_match  194 non-null    object 
 3   pattern_matched           194 non-null    object 
 4   part                      194 non-null    object 
 5   start_offset              194 non-null    float64
 6   entry_number              194 non-null    int64  
 7   sum_durs                  194 non-null    float64
 8   offset_diffs              194 non-null    object 
 9   predicted_type            194 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 21.7+ KB
